In [3]:
%pip install torch
%pip install torchdata
%pip install numpy
%pip intsall transformers
%pip install datasets
%pip install rouge_score
%pip install evaluate
%pip install loralib
%pip install peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [4]:
import pandas as pd
import numpy as np
import torch
import time
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import evaluate

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [5]:
model_name = "google/flan-t5-base"

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype = torch.bfloat16)


# Tokenization

tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [12]:
def get_num_trainable_params(model):
  total_params = 0
  total_trainable_params = 0

  for _ , params in model.named_parameters():
    total_params += params.numel()

    if params.requires_grad:
      total_trainable_params += params.numel()

  return f"Trainable Param = {total_trainable_params}\nTotal Params = {total_params}\n% of trainable params = {100*(total_trainable_params/total_params)}"

print(get_num_trainable_params(original_model))

Trainable Param = 3538944
Total Params = 251116800
% of trainable params = 1.4092820552029972


In [6]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [7]:
def tokenize_function(example):
  start_prompt = "Symmarize following conversations:\n"
  end_prompt = "\nSumamry:\n"
  prompt = [start_prompt+dialogue+end_prompt for dialogue in example['dialogue']]
  example['input_ids'] = tokenizer(prompt, padding= 'max_length', truncation = True, return_tensors='pt').input_ids
  example['labels'] = tokenizer(example['summary'], padding= 'max_length', truncation = True, return_tensors='pt').input_ids

  return example


tokenized_dataset = dataset.map(tokenize_function, batched = True)
tokenized_dataset


Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'labels'],
        num_rows: 1500
    })
})

In [8]:
tokenized_dataset = tokenized_dataset.remove_columns(['id', 'dialogue', 'summary', 'topic'])
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 500
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1500
    })
})

In [9]:
print(f"Train data shape = {tokenized_dataset['train'].shape}")
print(f"validation data shape = {tokenized_dataset['validation'].shape}")
print(f"Test data shape = {tokenized_dataset['test'].shape}")

Train data shape = (12460, 2)
validation data shape = (500, 2)
Test data shape = (1500, 2)


In [10]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=['q',
                    'v',],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type=TaskType.SEQ_2_SEQ_LM   #FLAN-T5
)


In [13]:
peft_model = get_peft_model(original_model, lora_config)

print(get_num_trainable_params(peft_model))

Trainable Param = 3538944
Total Params = 251116800
% of trainable params = 1.4092820552029972


In [23]:
output_dir = f'./peft-lora-dialogsum'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate = 1e-3,
    num_train_epochs = 2,
    logging_steps = 100,)

peft_trainer = Trainer(model = peft_model,
                      args = peft_training_args,
                      train_dataset = tokenized_dataset['train'])

In [24]:
# peft_trainer.train()

peft_trainer.train(resume_from_checkpoint=True)
peft_model_path =  "./peft-lora-dialogsum-checkpoints"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

	logging_steps: 100 (from args) != 1 (from trainer_state.json)


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('./peft-lora-dialogsum-checkpoints/tokenizer_config.json',
 './peft-lora-dialogsum-checkpoints/special_tokens_map.json',
 './peft-lora-dialogsum-checkpoints/spiece.model',
 './peft-lora-dialogsum-checkpoints/added_tokens.json',
 './peft-lora-dialogsum-checkpoints/tokenizer.json')

In [16]:
device = original_model.device

dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    print(idx)
    prompt = f"""
    Summarize the following conversations :

    {dialogue}

    Summary :
    """
    input_ids = tokenizer(prompt, return_tensors = 'pt').input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_output = original_model.generate(input_ids = input_ids.to(device),
                                                generation_config = GenerationConfig(max_new_tokens = 50))
    original_model_text_output = tokenizer.decode(original_model_output[0], skip_special_tokens = True)

    peft_model_output = peft_model.generate(input_ids = input_ids.to(device),
                                                generation_config = GenerationConfig(max_new_tokens = 50))
    peft_model_text_output = tokenizer.decode(peft_model_output[0], skip_special_tokens = True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])

df

0
1
2
3
4
5
6
7
8
9


,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,#Person1#: I need to take a dictation for you....,#Person1#: I need to take a dictation for you....
1,In order to prevent employees from wasting tim...,#Person1#: I need to take a dictation for you....,#Person1#: I need to take a dictation for you....
2,Ms. Dawson takes a dictation for #Person1# abo...,#Person1#: I need to take a dictation for you....,#Person1#: I need to take a dictation for you....
3,#Person2# arrives late because of traffic jam....,The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
4,#Person2# decides to follow #Person1#'s sugges...,The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
5,#Person2# complains to #Person1# about the tra...,The traffic jam at the Carrefour intersection ...,The traffic jam at the Carrefour intersection ...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are getting divorced..............,Masha and Hero are getting divorced..............
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are getting divorced..............,Masha and Hero are getting divorced..............
8,#Person1# and Kate talk about the divorce betw...,Masha and Hero are getting divorced..............,Masha and Hero are getting divorced..............
9,#Person1# and Brian are at the birthday party ...,Brian's birthday is coming up.nel is going to ...,Brian's birthday is coming up.nel is going to ...


In [17]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('PEFT Model:')
print(original_model_results)
print('Original Model:')
print(peft_model_results)

PEFT Model:
{'rouge1': 0.2989165854375938, 'rouge2': 0.12392050696319232, 'rougeL': 0.25462131497443496, 'rougeLsum': 0.25467400271619833}
Original Model:
{'rouge1': 0.2989165854375938, 'rouge2': 0.12392050696319232, 'rougeL': 0.25462131497443496, 'rougeLsum': 0.25467400271619833}


In [46]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(original_model_results.values())) - np.array(list(peft_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: -4.345111447228772%
rouge2: -2.037882905991692%
rougeL: -0.485275165363086%
rougeLsum: -0.6199136291106577%


In [47]:
%pip install sacrebleu

In [48]:
import sacrebleu

original_model_references = human_baseline_summaries[0:len(original_model_summaries)]
peft_model_references = human_baseline_summaries[0:len(peft_model_summaries)]

original_model_bleu = sacrebleu.corpus_bleu(original_model_summaries, original_model_references)
peft_model_bleu = sacrebleu.corpus_bleu(peft_model_summaries, peft_model_references)

print('Original Model BLEU:', original_model_bleu.score)
print('PEFT Model BLEU:', peft_model_bleu.score)


Original Model BLEU: 0.1450037232836522
PEFT Model BLEU: 0.14375239311913163


In [49]:
!zip -r /content/peft-lora-dialogsum.zip /content/peft-lora-dialogsum
!zip -r /content/peft-lora-dialogsum-checkpoints.zip /content/peft-lora-dialogsum-checkpoints



updating: content/peft-lora-dialogsum/ (stored 0%)
updating: content/peft-lora-dialogsum/config.json (deflated 62%)
updating: content/peft-lora-dialogsum/checkpoint-3000/ (stored 0%)
updating: content/peft-lora-dialogsum/checkpoint-3000/trainer_state.json (deflated 89%)
updating: content/peft-lora-dialogsum/checkpoint-3000/optimizer.pt (deflated 23%)
updating: content/peft-lora-dialogsum/checkpoint-3000/README.md (deflated 66%)
updating: content/peft-lora-dialogsum/checkpoint-3000/adapter_config.json (deflated 52%)
updating: content/peft-lora-dialogsum/checkpoint-3000/rng_state.pth (deflated 25%)
updating: content/peft-lora-dialogsum/checkpoint-3000/adapter_model.safetensors (deflated 22%)
updating: content/peft-lora-dialogsum/checkpoint-3000/training_args.bin (deflated 51%)
updating: content/peft-lora-dialogsum/checkpoint-3000/scheduler.pt (deflated 56%)
updating: content/peft-lora-dialogsum/checkpoint-2000/ (stored 0%)
updating: content/peft-lora-dialogsum/checkpoint-2000/trainer_sta

In [1]:
from google.colab import files
files.download('/content/peft-lora-dialogsum.zip')
files.download('/content/peft-lora-dialogsum-checkpoints.zip')

FileNotFoundError: Cannot find file: /content/peft-lora-dialogsum.zip